In [1]:
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from typing import Annotated
from typing_extensions import TypedDict
import getpass
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langchain_mcp_adapters.tools import load_mcp_tools
import os
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()


In [2]:
server_params = StdioServerParameters(
    command="python",
    args=[
        "D://fundaura-chatbot//mcp_server.py"
      ]
)

In [3]:
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("GROQ_API_KEY")

_set_env("TAVILY_API_KEY")


In [3]:
# del graph_builder
class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

In [4]:
from langchain_groq import ChatGroq
llm = ChatGroq(model = "qwen-2.5-32b")
from langchain_community.tools.tavily_search import TavilySearchResults


# tavilytool = TavilySearchResults(max_results=2)
# mcp_tool =load_mcp_tools()
# tools = [tool]
# llm_with_tools = llm.bind_tools(tools)

In [5]:
def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

# The first argument is the unique node name
# The second argument is the function or object that will be called whenever
# the node is used.
graph_builder.add_node("chatbot", chatbot)


In [6]:
import json

from langchain_core.messages import ToolMessage


class BasicToolNode:
    """A node that runs the tools requested in the last AIMessage."""

    def __init__(self, tools: list) -> None:
        self.tools_by_name = {tool.name: tool for tool in tools}

    def __call__(self, inputs: dict):
        if messages := inputs.get("messages", []):
            message = messages[-1]
        else:
            raise ValueError("No message found in input")
        outputs = []
        for tool_call in message.tool_calls:
            tool_result = self.tools_by_name[tool_call["name"]].invoke(
                tool_call["args"]
            )
            outputs.append(
                ToolMessage(
                    content=json.dumps(tool_result),
                    name=tool_call["name"],
                    tool_call_id=tool_call["id"],
                )
            )
            print(
                f"Tool call {tool_call['id']} result: {tool_result}"
            )
        return {"messages": outputs}


tool_node = BasicToolNode(tools=[tool])
graph_builder.add_node("my_tools", tool_node)

In [7]:
def route_tools(
    state: State,
):
    """
    Use in the conditional_edge to route to the ToolNode if the last message
    has tool calls. Otherwise, route to the end.
    """
    if isinstance(state, list):
        ai_message = state[-1]
    elif messages := state.get("messages", []):
        ai_message = messages[-1]
    else:
        raise ValueError(f"No messages found in input state to tool_edge: {state}")
    if hasattr(ai_message, "tool_calls") and len(ai_message.tool_calls) > 0:
        return "tools"
    return END


# The `tools_condition` function returns "tools" if the chatbot asks to use a tool, and "END" if
# it is fine directly responding. This conditional routing defines the main agent loop.
graph_builder.add_conditional_edges(
    "chatbot",
    route_tools,
    # The following dictionary lets you tell the graph to interpret the condition's outputs as a specific node
    # It defaults to the identity function, but if you
    # want to use a node named something else apart from "tools",
    # You can update the value of the dictionary to something else
    # e.g., "tools": "my_tools"
    {"tools": "my_tools", END: END},
)
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("my_tools", "chatbot")
graph_builder.add_edge(START, "chatbot")
graph = graph_builder.compile(checkpointer=memory)

In [8]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [41]:
def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [{"role": "user", "content": user_input}]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)


while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break
        stream_graph_updates(user_input)
    except:
        # fallback if input() is not available
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break

Goodbye!


In [45]:
def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [{"role": "user", "content": user_input}]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)


while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break
        stream_graph_updates(user_input)
    except:
        # fallback if input() is not available
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break

Assistant: Hello! How can I assist you today?
Assistant: It seems there might have been a typo in your message. If you're asking whether I have tools available, yes, I do have access to certain tools that can assist with your queries. For example, I can perform web searches to find comprehensive, accurate, and trusted results. If you need help with something specific, please let me know how I can assist you!
Assistant: Yes, I have access to a few tools that can help me gather information or perform tasks. For instance, I can use a search engine to find information on the internet. Would you like to use any specific tool to find an answer to a question or perform a task? Please let me know how I can assist you.
Assistant: 
Tool call call_sb6e result: [{'title': 'Chennimalai - Wikipedia', 'url': 'https://en.wikipedia.org/wiki/Chennimalai', 'content': 'Elevation | 330\xa0m (1,080\xa0ft)\nPopulation(2001)\n•\xa0Total | 30,000 (est.)\nLanguages\n•\xa0Official | Tamil\nTime zone | UTC+5:30(I

In [9]:
config = {"configurable": {"thread_id": "1"}}

In [10]:
user_input = "Hi there! My name is Will."

# The config is the **second positional argument** to stream() or invoke()!
events = graph.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    config,
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Hi there! My name is Will.
================================== Ai Message ==================================

Hello, Will! It's nice to meet you. How can I assist you today?


In [11]:
user_input = "Remember my name?"

# The config is the **second positional argument** to stream() or invoke()!
events = graph.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    config,
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Remember my name?
================================== Ai Message ==================================

Yes, I remember your name is Will. How can I assist you further?


TypeError: 'generator' object is not subscriptable

In [10]:
from db_utilities.mongo_implementation import MongoImplement

db = MongoImplement(
        connection_string="mongodb://localhost:27017/",
        db_name="fundaura",
        max_pool=5,
        server_selection_timeout=60000
    )

results = db.read(collection_name="transactions", query = {'aggregate': [{'$match': {'date': {'$gte': '2025-02-19T00:00:00.000Z', '$lt': '2025-02-20T00:00:00.000Z'}}}, {'$group': {'_id': None, 'totalAmount': {'$sum': '$amount'}}}]})

2025-04-20 23:03:43,420 - db_utilities.mongo_implementation - INFO - Initializing connection pool for database connection, with fundaura
2025-04-20 23:03:43,429 - db_utilities.mongo_implementation - INFO - Made 5 max_connections with fundaura
2025-04-20 23:03:43,430 - db_utilities.mongo_implementation - INFO - Fetching documents from transactions || query = {'aggregate': [{'$match': {'date': {'$gte': '2025-02-19T00:00:00.000Z', '$lt': '2025-02-20T00:00:00.000Z'}}}, {'$group': {'_id': None, 'totalAmount': {'$sum': '$amount'}}}]}


In [ ]:
from datetime import datetime, timedelta

start_date = datetime.fromisoformat("2025-02-19")
end_date = start_date + timedelta(days=1)

pipeline = [
    {
        "$match": {
            "date": {
                "$gte": start_date,
                "$lt": end_date
            }
        }
    },
    {
        "$group": {
            "_id": None,
            "totalAmount": {"$sum": "$amount"}
        }
    }
]

result = db.client.transactions.aggregate([{'$match': {'date': {'$gte': '2025-02-19T00:00:00.000Z', '$lt': '2025-02-20T00:00:00.000Z'}}}, {'$group': {'_id': None, 'totalAmount': {'$sum': '$amount'}}}])


OperationFailure: {aggregate: 1} is not valid for '$match'; a collection is required., full error: {'ok': 0.0, 'errmsg': "{aggregate: 1} is not valid for '$match'; a collection is required.", 'code': 73, 'codeName': 'InvalidNamespace'}

In [33]:
transactions_collection = db.client["fundaura"]["transactions"]
result = transactions_collection.aggregate(
    [{'$match': {'date': {'$gte': f"{start_date.__str__()}", '$lt': f"{end_date}"}}}, {'$group': {'_id': None, 'totalAmount': {'$sum': '$amount'}}}]
)

In [34]:
f"{start_date}"

'2025-02-19 00:00:00'

In [35]:
list(result)

[]

In [9]:
# Create server parameters for stdio connection
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent
import asyncio
import nest_asyncio 
nest_asyncio.apply()

model = llm

server_params =  StdioServerParameters(
    command="python",
    # Make sure to update to the full absolute path to your math_server.py file
    args=["D://fundaura-chatbot//mcp_server.py"],
)

async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        # Initialize the connection
        await session.initialize()

        # Get tools
        tools = await load_mcp_tools(session)

        # Create and run the agent
        agent = create_react_agent(model, tools)
        agent_response = await agent.ainvoke({"messages": "give number no of documents in transactions collection"})

NotImplementedError: 